<a href="https://colab.research.google.com/github/brooklinsantosh/HackLive-3-Guided-Hackathon---NLP/blob/master/DataHack3NLP_SimpleTransformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPU check

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


# Loading Libraries

In [ ]:
!pip install simpletransformers

      Successfully uninstalled ipykernel-4.10.1


In [ ]:
import numpy as np
import pandas as pd

from simpletransformers.classification import MultiLabelClassificationModel, MultiLabelClassificationArgs

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score

import wandb

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


# Data

In [ ]:
TARGET_COLS = ['Analysis of PDEs', 'Applications',
               'Artificial Intelligence', 'Astrophysics of Galaxies',
               'Computation and Language', 'Computer Vision and Pattern Recognition',
               'Cosmology and Nongalactic Astrophysics',
               'Data Structures and Algorithms', 'Differential Geometry',
               'Earth and Planetary Astrophysics', 'Fluid Dynamics',
               'Information Theory', 'Instrumentation and Methods for Astrophysics',
               'Machine Learning', 'Materials Science', 'Methodology', 'Number Theory',
               'Optimization and Control', 'Representation Theory', 'Robotics',
               'Social and Information Networks', 'Statistics Theory',
               'Strongly Correlated Electrons', 'Superconductivity',
               'Systems and Control']

TOPIC_COLS = ['Computer Science', 'Mathematics', 'Physics', 'Statistics']

In [ ]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
ss = pd.read_csv('SampleSubmission_Uqu2HVA.csv')

train['labels'] = list(zip(train['Analysis of PDEs'].tolist(), train['Applications'].tolist(), train['Artificial Intelligence'].tolist(), 
                           train['Astrophysics of Galaxies'].tolist(),  train['Computation and Language'].tolist(), train['Computer Vision and Pattern Recognition'].tolist(),
                           train['Cosmology and Nongalactic Astrophysics'].tolist(), train['Data Structures and Algorithms'].tolist(), train['Differential Geometry'].tolist(),
                           train['Earth and Planetary Astrophysics'].tolist(), train['Fluid Dynamics'].tolist(), train['Information Theory'].tolist(),
                           train['Instrumentation and Methods for Astrophysics'].tolist(), train['Machine Learning'], train['Materials Science'].tolist(),
                           train['Methodology'].tolist(), train['Number Theory'].tolist(), train['Optimization and Control'].tolist(),
                           train['Representation Theory'].tolist(), train['Robotics'].tolist(), train['Social and Information Networks'].tolist(),
                           train['Statistics Theory'].tolist(), train['Strongly Correlated Electrons'].tolist(), 
                           train['Superconductivity'].tolist(), train['Systems and Control'].tolist()))

topics = []
for i in range(train.shape[0]):
    topic = ''
    for j in TOPIC_COLS:
        if(train.iloc[i][j]==1):
            topic += (j + ' ')
    topics.append(topic)

train['text'] = topics + train['ABSTRACT']

trn, val = train_test_split(train, test_size=0.2, random_state=2)

leng =[]
for i in range(train.shape[0]):
    l = len((train['text'].iloc[i]).split(' '))
    leng.append(l)

train['len'] = leng

topics_test = []
for i in range(test.shape[0]):
    topic = ''
    for j in TOPIC_COLS:
        if(test.iloc[i][j]==1):
            topic += (j + ' ')
    topics_test.append(topic)

tst= pd.DataFrame((topics_test + test['ABSTRACT']))
tst.columns = ['text']

# Helper Methods

In [ ]:
def get_best_thresholds(true, preds):
  thresholds = [i/100 for i in range(100)]
  best_thresholds = []
  for idx in range(25):
    f1_scores = [f1_score(true[:, idx], (preds[:, idx] > thresh) * 1) for thresh in thresholds]
    best_thresh = thresholds[np.argmax(f1_scores)]
    best_thresholds.append(best_thresh)
  return best_thresholds

In [ ]:
def download_preds(preds_test, file_name = 'hacklive_sub.csv'):
  ss[TARGET_COLS] = preds_test
  ss.to_csv(file_name, index = False)
  from google.colab import files
  files.download(file_name)

# Configs

In [ ]:
model_args = MultiLabelClassificationArgs()

#model_args.use_early_stopping = False
#model_args.early_stopping_delta = 0.01
#model_args.early_stopping_metric = "eval_loss"
#model_args.early_stopping_metric_minimize = True
#model_args.early_stopping_patience = 3
#model_args.evaluate_during_training_steps = 1000

model_args.eval_batch_size = 16
model_args.evaluate_during_training = True
model_args.evaluate_during_training_silent = False
model_args.evaluate_during_training_steps = 1000
model_args.learning_rate = 0.00002177
model_args.manual_seed = 4
model_args.max_seq_length = 320
model_args.multiprocessing_chunksize = 5000
model_args.no_cache = True
model_args.no_save = True
model_args.num_train_epochs = 16
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.train_batch_size = 8
model_args.gradient_accumulation_steps = 2
model_args.train_custom_parameters_only = False
model_args.save_eval_checkpoints = False
model_args.save_model_every_epoch = False

# Model Training

In [ ]:
model = MultiLabelClassificationModel('roberta', 'roberta-base', use_cuda=True, num_labels=25, args=model_args)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'clas

In [ ]:
model.train_model(trn,eval_df=val)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


(1050,
 {'LRAP': [0.5204119642260323,
   0.7872954423500944,
   0.8561043547075311,
   0.8758990568121239,
   0.8835648490628084,
   0.8889730814685808,
   0.8899924285056316],
  'eval_loss': [0.17330125262114135,
   0.11765960485420444,
   0.094247703271156,
   0.08338556582616134,
   0.07878231443464756,
   0.07724176918749105,
   0.0764775238931179],
  'global_step': [175, 350, 525, 700, 875, 1000, 1050],
  'train_loss': [0.22723247110843658,
   0.14886417984962463,
   0.15509793162345886,
   0.11223648488521576,
   0.03757068142294884,
   0.06785576045513153,
   0.051411498337984085]})

# Evaluation & Best threshold

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(val)
print(result)


{'LRAP': 0.810255241752497, 'eval_loss': 0.11467986807904461}


In [ ]:
best_thresholds = get_best_thresholds(val[TARGET_COLS].values, model_outputs)

for i, thresh in enumerate(best_thresholds):
  model_outputs[:, i] = (model_outputs[:, i] > thresh) * 1
  
f1_score(val[TARGET_COLS], model_outputs, average='micro')

0.6959434313360625

# Prediction

In [ ]:
predictions, raw_outputs = model.predict(tst['text'])

KeyboardInterrupt: ignored

In [ ]:
for i, thresh in enumerate(best_thresholds):
  raw_outputs[:, i] = (raw_outputs[:, i] > thresh) * 1

download_preds(raw_outputs, 'hacklive_roberta_large_3ep_threshold_optimized.csv')